<a href="https://colab.research.google.com/github/rubaahmedkhan/AgenticAI-Toolkit-/blob/main/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  -qU openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, ModelSettings, function_tool
from agents.run import RunConfig
from google.colab import userdata


In [64]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
model_settings = ModelSettings(
    max_tokens=50,
    # workflow_name="agent flow",     # not write workflow name here

)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    model_settings=model_settings,
    workflow_name="agent flow",
    tracing_disabled=False,
)

In [5]:
agent = Agent(
    name="",
    instructions="you are a helpful assistant",

)

In [54]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

agent = Agent(
    name="agent",   # 123-> pass , ""->pass ,gyhuf-> NameError
    instructions="Always respond in haiku form",#Instruction ->Type error
    model="o3-mini",
    model_settings=ModelSettings(max_tokens=50,
                                 tool_choice="required",

                                 ),
    tools=[get_weather],# tool -> TypeError, without define tool pass in list->NameError
    reset_tool_choice=False,
    #tool_use_behavior="one_shot", # UserError
    #tool_use_behavior="stop_on_first_tool"   loop mai nahi phuncyga

)

result = await Runner.run(agent, "what is the weather in london",run_config=config)
print(result.final_output)

MaxTurnsExceeded: Max turns (10) exceeded

In [55]:

agent = Agent(
    name="NumberAgent",
    instructions="helpfull assistant.",
    output_type=int  # <- Sirf integer output accept karega
)

result = await Runner.run(agent, "what is ai",run_config=config)   # without input-> type error
print(result.final_output)



123


In [58]:
async def main():
    agent = Agent(name="Assistant", instructions="Reply very concisely.")

    with trace(workflow_name="Conversation", group_id=thread_id):
        # First turn
        result = await Runner.run(agent, "What city is the Golden Gate Bridge in?")
        print(result.final_output)
        # San Francisco

        # Second turn
        new_input = result.to_input_list() + [{"role": "user", "content": "What state is it in?"}]
        result = await Runner.run(agent, new_input)
        print(result.final_output)
        # California

In [69]:
from pydantic import BaseModel
from agents import Agent

# Step 1: Define output type
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# Step 2: Create agent with structured output
agent = Agent(
    name="Calendar extractor",
    instructions="Extract calendar events from the input. Make sure to identify the event name, date, and participants.",
    output_type=CalendarEvent,
)

# Step 3: Sample input text
text = "There is a meeting called 'Team Sync' on July 1st, 2025. John and Sara will attend."

# Step 4: Run agent and get structured output
result = await Runner.run(agent, text,run_config=config)
print(result.final_output)     # without await -> AttributeError , syn mai await TypeError
print("---------input list-----------")
print(result.to_input_list())
print("------------last agent----------")
result.last_agent


name='Team Sync' date='July 1st, 2025' participants=['John', 'Sara']
---------input list-----------
[{'content': "There is a meeting called 'Team Sync' on July 1st, 2025. John and Sara will attend.", 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '{\n  "date": "July 1st, 2025",\n  "name": "Team Sync",\n  "participants": ["John", "Sara"]\n}', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}]
------------last agent----------


Agent(name='Calendar extractor', instructions='Extract calendar events from the input. Make sure to identify the event name, date, and participants.', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=<class '__main__.CalendarEvent'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [38]:
from pydantic import BaseModel
from agents import Agent, Runner
import asyncio

# Output model (Pydantic still needed for output structuring)
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# Create agent (no input_type defined)
agent = Agent(
    name="DictInputAgent",
    instructions="You will receive a dict with a 'text' field. Extract the calendar event (name, date, participants).",
    output_type=CalendarEvent,
)

# Input as dict (NOT Pydantic model)
input_data = {
    "text": "The event 'Tech Summit' is on September 10, 2025, with participants Emma and Noah."
}

# Async run
async def main():
    result = await Runner.run(agent, input_data,run_config=config)
    print(result.final_output)

asyncio.run(main())


AttributeError: 'dict' object has no attribute 'extend'

In [42]:
from pydantic import BaseModel
from agents import RunContextWrapper
class UserContext(BaseModel):
    name: str


async def dynamic_instructions(                   # sync, async dono function chalskty hain
    context: RunContextWrapper[UserContext], agent: Agent[UserContext]
) -> str:
    return f"The user's name is {context.context.name}. Help them with their questions."


agent = Agent[UserContext](
    name="Triage agent",
    instructions=dynamic_instructions,
)